In [1]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

#

# Code:

In [2]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # [mm]
BlockLength = 174  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


89.0 x 174 x 19.0


In [3]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

cnc.reset()

(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [4]:
if "Alarm" in cnc.status:
    cnc.home()

In [5]:
cnc.init()

True

# Generate Program


In [6]:
def dot(X=10, Y=10, Z=0, power=1, dwell=1, **kwargs):
    program = GCode.GCode(**kwargs)
    
    program.G0(X=X, Y=Y, Z=Z)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.

    return program

In [30]:
spacings=[0.75, 1, 1.25]
spacings

[0.75, 1, 1.25]

In [31]:
powers = [25, 50, 75, 100, 150, 200, 255]
len(powers)

7

In [32]:
dwells = [0.1, 0.5, 0.75, 1, 1.5, 2, 2.5]
len(dwells)

7

In [ ]:
cnc.init()
margin=5 # [mm]
test_len=5 # [mm]
for idx0, power in enumerate(powers):
    X0_next=margin # [mm]
    Y = margin + idx0*2.5 # [mm]
    assert(Y<BlockHeight)
    for idx1, dwell in enumerate(dwells):
        for idx2, spacing in enumerate(spacings):
            X0=X0_next+(idx2*(test_len)) #[mm]
            Xf=X0+test_len #[mm]
            assert(Xf<BlockLength)
            XX=np.arange(X0, Xf, spacing)
            print("Laserin({}:{}:{}, {}, {}, {})".format(X0, spacing, Xf, Y, power, dwell), end="")
            for X in XX:
                pass
                program = dot(X=X, Y=Y, Z=-14, power=power, dwell=dwell)
                cnc.run(program)
            print("... Done")
        X0_next = Xf+1

In [29]:
spacings

[0.5, 1, 1.5]

In [35]:
powers

[25, 50, 75, 100, 150, 200]

In [36]:
dwells

[0.1, 0.5]